In [154]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
from dataclasses import dataclass
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

# from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
# from sklearn.decomposition import PCA
# from sklearn.ensemble import IsolationForest

import warnings
warnings.filterwarnings("ignore")

# hv.renderer('bokeh').theme = 'dark_minimal'


In [155]:
# dataset_root = Path(r"C:\Users\Turquin\Documents\MLFPMA - Machine Learning for Predictive Maintenance Application\Project\Dataset") # Raw string works without escaping \
dataset_root = Path("./Dataset")

@dataclass
class Case():
    info: pd.DataFrame
    measurements: pd.DataFrame


class RawDataset():
    def __init__(self, root, unit = "VG4", load_training=False, load_synthetic=False, load_anomalies=False) -> None:
        read_pq_file = lambda f: pq.read_table(root / f).to_pandas()
        
        cases = {
            "test": [f"{unit}_generator_data_testing_real_measurements.parquet", root / f"{unit}_generator_data_testing_real_info.csv" ], 
        }

        if load_training:
            cases = {
                **cases,
                "train": [f"{unit}_generator_data_training_measurements.parquet", root / f"{unit}_generator_data_training_info.csv" ], 
            }

        if load_synthetic:
            cases = {
                **cases,
                "test_s01": [f"{unit}_generator_data_testing_synthetic_01_measurements.parquet", root / f"{unit}_generator_data_testing_synthetic_01_info.csv"], 
                "test_s02": [f"{unit}_generator_data_testing_synthetic_02_measurements.parquet", root / f"{unit}_generator_data_testing_synthetic_02_info.csv"]
            }

        if load_anomalies:
            anomaly_folder = Path("synthetic_anomalies")  # Relative path
            subdataset = ["01", "02"]
            anomaly_types = ["a", "b", "c"]
            for anomaly in subdataset:
                for subtype in anomaly_types:
                    anomaly_key = f"anomaly_{anomaly}_type_{subtype}"
                    anomaly_file = f"{unit}_anomaly_{anomaly}_type_{subtype}.parquet"
                    full_anomaly_path = root / anomaly_folder / anomaly_file
                    if full_anomaly_path.exists():
                        cases[anomaly_key] = [anomaly_folder / anomaly_file, None]

        
        self.data_dict = dict()
        
        for id_c, c in cases.items():
            # if you need to verify the parquet header:
            # pq_rows = RawDataset.read_parquet_schema_df(root / c[0])
            measurements = read_pq_file(c[0])
            info = pd.read_csv(c[1]) if c[1] is not None else None
            self.data_dict[id_c] = Case(info, measurements)
    
    @staticmethod
    def read_parquet_schema_df(uri: str) -> pd.DataFrame:
        """Return a Pandas dataframe corresponding to the schema of a local URI of a parquet file.

        The returned dataframe has the columns: column, pa_dtype
        """
        # Ref: https://stackoverflow.com/a/64288036/
        schema = pq.read_schema(uri, memory_map=True)
        schema = pd.DataFrame(({"column": name, "pa_dtype": str(pa_dtype)} for name, pa_dtype in zip(schema.names, schema.types)))
        schema = schema.reindex(columns=["column", "pa_dtype"], fill_value=pd.NA)  # Ensures columns in case the parquet file has an empty dataframe.
        return schema
    

rds_u4 = RawDataset(dataset_root, "VG4", load_synthetic=False, load_training=True)
rds_u5 = RawDataset(dataset_root, "VG5", load_synthetic=True, load_training=True, load_anomalies=True)
rds_u6 = RawDataset(dataset_root, "VG6", load_synthetic=True, load_training=True, load_anomalies=True)

In [156]:
def add_anomaly_ground_truth(rds):
    subdataset = ["01", "02"]
    anomaly_types = ["a", "b", "c"]

    results = []
    for anomaly in subdataset:
        test_s012 = rds.data_dict[f'test_s{anomaly}'].measurements

        for subtype in anomaly_types:
            anomaly_key = f"anomaly_{anomaly}_type_{subtype}"
            labeled_df = rds.data_dict[anomaly_key].measurements
            test_s012.loc[labeled_df['ground_truth'] == 1, anomaly_key] = 1
        
        test_s012['anomaly'] = (test_s012[[f'anomaly_{anomaly}_type_a',f'anomaly_{anomaly}_type_b',f'anomaly_{anomaly}_type_c']].max(axis=1) == 1).astype(int)
        results.append(test_s012)

    return results

u5_s01, u5_s02 = add_anomaly_ground_truth(rds_u5)
u6_s01, u6_s02 = add_anomaly_ground_truth(rds_u6)

In [157]:
def anomaly_na(df):
    cols_anomaly = [col for col in df.columns if 'anomaly' in col]
    df[cols_anomaly] = df[cols_anomaly].fillna(0)
    return df 

anomaly_na(u5_s01)
anomaly_na(u5_s02)
anomaly_na(u6_s01)
anomaly_na(u6_s02)


,tot_activepower,ext_tmp,plant_tmp,charge,coupler_position,injector_01_opening,injector_02_opening,injector_03_opening,injector_04_opening,injector_05_opening,...,equilibrium_turbine_mode,dyn_only_on,pump_mode,short_circuit_mode,equilibrium_pump_mode,equilibrium_short_circuit_mode,anomaly_02_type_a,anomaly_02_type_b,anomaly_02_type_c,anomaly
2021-11-01 00:00:00+01:00,0.000000,14.845771,18.216559,4.311914,11.803000,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,0.0,0.0,0.0,0
2021-11-01 00:00:30+01:00,0.000000,14.852612,18.233796,4.464852,11.803000,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,0.0,0.0,0.0,0
2021-11-01 00:01:00+01:00,0.000000,14.859453,18.251033,4.445999,11.803000,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,0.0,0.0,0.0,0
2021-11-01 00:01:30+01:00,0.000000,14.866293,18.268270,4.140497,11.803000,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,0.0,0.0,0.0,0
2021-11-01 00:02:00+01:00,0.000000,14.873134,18.285507,4.453990,11.803000,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-30 23:58:00+01:00,-120.287556,10.201256,16.065450,-1.854672,185.406032,0.0,0.0,0.0,0.0,0.0,...,False,False,True,False,True,False,0.0,0.0,0.0,0
2021-12-30 23:58:30+01:00,-76.832558,10.203748,16.010313,-1.785339,185.403888,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,True,0.0,0.0,0.0,0
2021-12-30 23:59:00+01:00,0.000000,10.206240,15.955176,4.045595,185.401744,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,0.0,0.0,0.0,0
2021-12-30 23:59:30+01:00,0.000000,10.208732,15.900039,4.116665,185.399600,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,0.0,0.0,0.0,0


In [158]:
def get_control_vars(df):
    return df[(df['control_signal'] == True) | (df['input_feature'] == True)].attribute_name.values

u4_control_vars = get_control_vars(rds_u4.data_dict["train"].info)
u5_control_vars = get_control_vars(rds_u5.data_dict["train"].info)
u6_control_vars = get_control_vars(rds_u6.data_dict["train"].info)

u5_control_vars

array(['tot_activepower', 'charge', 'coupler_position',
       'injector_01_opening', 'injector_02_opening',
       'injector_03_opening', 'injector_04_opening',
       'injector_05_opening', 'pump_calculated_flow',
       'pump_pressure_diff', 'pump_rotspeed', 'turbine_pressure',
       'turbine_rotspeed', 'water_primary_pump_01_opening',
       'water_primary_pump_02_opening', 'timer_turbine_on_off',
       'timer_injector_opening'], dtype=object)

Separate dataframes by operating conditions and remove unnecessary columns about operating conditions.

In [159]:
def get_operating_modes(df):
    df_equilibrium_turbine_mode = df[df['equilibrium_turbine_mode'] == True]
    df_equilibrium_pump_mode = df[df['equilibrium_pump_mode'] == True]

    operating_cond_cols = ['machine_on', 'turbine_mode', 'all',
       'equilibrium_turbine_mode', 'dyn_only_on', 'pump_mode',
       'equilibrium_pump_mode']
    
    if 'short_circuit_mode' in df.columns:
        operating_cond_cols += ['short_circuit_mode', 'equilibrium_short_circuit_mode']
    
    df_equilibrium_turbine_mode = df_equilibrium_turbine_mode.drop(columns = operating_cond_cols)
    df_equilibrium_pump_mode = df_equilibrium_pump_mode.drop(columns = operating_cond_cols)

    return df_equilibrium_turbine_mode, df_equilibrium_pump_mode


In [160]:
# train sets
u4_train_equil_turbine, u4_train_equil_pump = get_operating_modes(rds_u4.data_dict["train"].measurements)
u5_train_equil_turbine, u5_train_equil_pump = get_operating_modes(rds_u5.data_dict["train"].measurements)
u6_train_equil_turbine, u6_train_equil_pump = get_operating_modes(rds_u6.data_dict["train"].measurements)

# synethetic test sets
u5_s01_equil_turbine, u5_s01_equil_pump = get_operating_modes(u5_s01)
u5_s02_equil_turbine, u5_s02_equil_pump = get_operating_modes(u5_s02)
u6_s01_equil_turbine, u6_s01_equil_pump = get_operating_modes(u6_s01)
u6_s02_equil_turbine, u6_s02_equil_pump = get_operating_modes(u6_s02)

# real test sets
u4_test_equil_turbine, u4_test_equil_pump = get_operating_modes(rds_u4.data_dict["test"].measurements)
u5_test_equil_turbine, u5_test_equil_pump = get_operating_modes(rds_u5.data_dict["test"].measurements)
u6_test_equil_turbine, u6_test_equil_pump = get_operating_modes(rds_u6.data_dict["test"].measurements)


In [161]:
list_dfs = [u4_train_equil_turbine, u4_train_equil_pump, u5_train_equil_turbine, u5_train_equil_pump, u6_train_equil_turbine, u6_train_equil_pump,
            u5_s01_equil_turbine, u5_s01_equil_pump, u5_s02_equil_turbine, u5_s02_equil_pump,
            u6_s01_equil_turbine, u6_s01_equil_pump, u6_s02_equil_turbine, u6_s02_equil_pump,
            u4_test_equil_turbine, u4_test_equil_pump, u5_test_equil_turbine, u5_test_equil_pump, u6_test_equil_turbine, u6_test_equil_pump]

u4_dfs = [u4_train_equil_turbine,
    u4_train_equil_pump,
    u4_test_equil_turbine,
    u4_test_equil_pump]

u56_dfs = [u5_train_equil_turbine,
    u5_train_equil_pump,
    u5_s01_equil_turbine,
    u5_s01_equil_pump,
    u5_s02_equil_turbine,
    u5_s02_equil_pump,
    u5_test_equil_turbine,
    u5_test_equil_pump,
    u6_train_equil_turbine,
    u6_train_equil_pump,
    u6_s01_equil_turbine,
    u6_s01_equil_pump,
    u6_s02_equil_turbine,
    u6_s02_equil_pump,
    u6_test_equil_turbine,
    u6_test_equil_pump]

Standardize columns

In [162]:
cols_to_scale = [col for col in u5_s01_equil_turbine.columns if ('anomaly' not in col and 'machine' not in col)]

In [163]:
scaler = StandardScaler()
scaled_dfs_u4 = [pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index) for df in u4_dfs]

(
    u4_train_equil_turbine,
    u4_train_equil_pump,
    u4_test_equil_turbine,
    u4_test_equil_pump
) = scaled_dfs_u4



final_scaled_dfs = []
for df in u56_dfs:
    scaled_dfs = scaler.fit_transform(df[cols_to_scale])
    df[cols_to_scale] = scaled_dfs
    final_scaled_dfs.append(df)

(
    u5_train_equil_turbine,
    u5_train_equil_pump,
    u5_s01_equil_turbine,
    u5_s01_equil_pump,
    u5_s02_equil_turbine,
    u5_s02_equil_pump,
    u5_test_equil_turbine,
    u5_test_equil_pump,
    u6_train_equil_turbine,
    u6_train_equil_pump,
    u6_s01_equil_turbine,
    u6_s01_equil_pump,
    u6_s02_equil_turbine,
    u6_s02_equil_pump,
    u6_test_equil_turbine,
    u6_test_equil_pump
) = final_scaled_dfs


In [165]:
u6_s01_equil_pump

,tot_activepower,ext_tmp,plant_tmp,charge,coupler_position,injector_01_opening,injector_02_opening,injector_03_opening,injector_04_opening,injector_05_opening,...,water_circ_hot_tmp,air_gap_negative_x_position,air_gap_positive_x_position,air_gap_negative_y_position,air_gap_positive_y_position,machine_off,anomaly_01_type_a,anomaly_01_type_b,anomaly_01_type_c,anomaly
2021-06-02 02:04:00+02:00,0.483461,-1.758137,-2.288500,-0.068941,-1.468817,0.0,0.0,-0.006367,-0.008616,-0.006367,...,-5.923976,4.364773,2.841183,4.076654,4.675421,False,1.0,0.0,0.0,1
2021-06-02 02:04:30+02:00,-0.408182,-1.758152,-2.287107,-0.102277,-1.289970,0.0,0.0,-0.006367,-0.008616,-0.006367,...,-5.894179,-1.254946,-0.294372,-0.338638,-0.001284,False,1.0,0.0,0.0,1
2021-06-02 02:05:00+02:00,-0.841667,-1.758166,-2.285715,-0.061231,-1.111123,0.0,0.0,-0.006367,-0.008616,-0.006367,...,-5.779258,0.692921,-0.214035,-0.270684,-0.038245,False,1.0,0.0,0.0,1
2021-06-02 02:05:30+02:00,-1.203102,-1.758180,-2.284322,-0.114406,-0.932276,0.0,0.0,-0.006367,-0.008616,-0.006367,...,-5.717088,-0.531678,-0.133698,-0.202730,-0.075206,False,1.0,0.0,0.0,1
2021-06-02 02:06:00+02:00,-1.312475,-1.758194,-2.282930,0.013089,-0.753429,0.0,0.0,-0.006367,-0.008616,-0.006367,...,-5.654918,0.737568,-0.053362,-0.134776,-0.112166,False,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-30 19:01:00+02:00,-2.239765,1.124337,3.234733,0.132139,1.130969,0.0,0.0,-0.006367,-0.008616,-0.006367,...,1.843063,-1.497971,3.597738,2.573343,2.412786,False,0.0,0.0,0.0,0
2021-07-30 19:01:30+02:00,-1.670996,1.130247,3.244945,-0.059156,1.130969,0.0,0.0,-0.006367,-0.008616,-0.006367,...,1.831104,-2.519837,3.842438,2.887934,2.668034,False,0.0,0.0,0.0,0
2021-07-30 19:02:00+02:00,-1.043516,1.136157,3.255156,-0.041600,1.130969,0.0,0.0,-0.006367,-0.008616,-0.006367,...,1.819145,1.258571,4.087139,3.202525,2.923282,False,0.0,0.0,0.0,0
2021-07-30 19:02:30+02:00,-1.798696,1.140814,3.265367,-0.078001,1.130969,0.0,0.0,-0.006367,-0.008616,-0.006367,...,1.807186,1.780178,4.331840,3.517116,3.178530,False,0.0,0.0,0.0,0


Copy this into your notebook to import the variables

In [164]:
'''
# training data
u4_train_equil_turbine = data_preprocessing.u4_train_equil_turbine
u4_train_equil_pump = data_preprocessing.u4_train_equil_pump
u5_train_equil_turbine = data_preprocessing.u5_train_equil_turbine
u5_train_equil_pump = data_preprocessing.u5_train_equil_pump
u6_train_equil_turbine = data_preprocessing.u6_train_equil_turbine
u6_train_equil_pump = data_preprocessing.u6_train_equil_pump

# synethetic test sets
u5_s01_equil_turbine = data_preprocessing.u5_s01_equil_turbine
u5_s01_equil_pump = data_preprocessing.u5_s01_equil_pump
u5_s02_equil_turbine = data_preprocessing.u5_s02_equil_turbine
u5_s02_equil_pump = data_preprocessing.u5_s02_equil_pump
u6_s01_equil_turbine = data_preprocessing.u6_s01_equil_turbine
u6_s01_equil_pump = data_preprocessing.u6_s01_equil_pump
u6_s02_equil_turbine = data_preprocessing.u6_s02_equil_turbine
u6_s02_equil_pump = data_preprocessing.u6_s02_equil_pump

# real test sets
u4_test_equil_turbine = data_preprocessing.u4_test_equil_turbine
u4_test_equil_pump = data_preprocessing.u4_test_equil_pump
u5_test_equil_turbine = data_preprocessing.u5_test_equil_turbine
u5_test_equil_pump = data_preprocessing.u5_test_equil_pump
u6_test_equil_turbine = data_preprocessing.u6_test_equil_turbine
u6_test_equil_pump = data_preprocessing.u6_test_equil_pump
'''

'\n# training data\nu4_train_equil_turbine = data_preprocessing.u4_train_equil_turbine\nu4_train_equil_pump = data_preprocessing.u4_train_equil_pump\nu5_train_equil_turbine = data_preprocessing.u5_train_equil_turbine\nu5_train_equil_pump = data_preprocessing.u5_train_equil_pump\nu6_train_equil_turbine = data_preprocessing.u6_train_equil_turbine\nu6_train_equil_pump = data_preprocessing.u6_train_equil_pump\n\n# synethetic test sets\nu5_s01_equil_turbine = data_preprocessing.u5_s01_equil_turbine\nu5_s01_equil_pump = data_preprocessing.u5_s01_equil_pump\nu5_s02_equil_turbine = data_preprocessing.u5_s02_equil_turbine\nu5_s02_equil_pump = data_preprocessing.u5_s02_equil_pump\nu6_s01_equil_turbine = data_preprocessing.u6_s01_equil_turbine\nu6_s01_equil_pump = data_preprocessing.u6_s01_equil_pump\nu6_s02_equil_turbine = data_preprocessing.u6_s02_equil_turbine\nu6_s02_equil_pump = data_preprocessing.u6_s02_equil_pump\n\n# real test sets\nu4_test_equil_turbine = data_preprocessing.u4_test_equil

Some examples:

For Unit 4:
- train on u4_train_equil_pump, test on u4_test_equil_pump

For Unit 5:
- train on u5_train_equil_turbine, test on u5_s01_equil_turbine, u5_s02_equil_turbine, u5_test_equil_turbine